In [ ]:
!git clone https://github.com/jerryjliu/gpt_index.git

In [ ]:
!pip install llama-index llama_hub wikipedia

In [3]:
import os
import openai
os.environ["OPENAI_API_KEY"] = "<open-ai-key>"
openai.api_key = os.environ["OPENAI_API_KEY"]

# Data connectors (LlamaHub)

In [4]:
from llama_hub.wikipedia.base import WikipediaReader

loader = WikipediaReader()
documents = loader.load_data(pages=['Berlin', 'Rome', 'Tokyo', 'Canberra', 'Santiago'])

# Basic query functionalities

In [5]:
print(len(documents))

5


In [6]:
print(documents[0])

Doc ID: 1bf24dc0-66fb-44bf-9a34-eec2c8666f78
Text: Berlin ( bur-LIN, German: [bɛʁˈliːn] ) is the capital and
largest city of Germany by both area and population. Its more than
3.85 million inhabitants make it the European Union's most populous
city, according to population within city limits. One of Germany's
sixteen constituent states, Berlin is surrounded by the State of
Brandenburg and contig...


In [7]:
from llama_index import VectorStoreIndex
# build an index over these Document objects.
index = VectorStoreIndex.from_documents(documents)
# you can query an index with the default QueryEngine
query_engine = index.as_query_engine()
response = query_engine.query("How many people live in Berlin")

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
print(response)

Berlin had a population of 3.75 million registered inhabitants at the end of 2018. However, the urban area of Berlin had about 4.5 million inhabitants in 2019, and the functional urban area was home to about 5.2 million people. The entire Berlin-Brandenburg capital region has a population of more than 6 million.


In [9]:
print(type(index))

<class 'llama_index.indices.vector_store.base.VectorStoreIndex'>


# Query Multiple Documents:
Source: https://gpt-index.readthedocs.io/en/latest/examples/usecases/10q_sub_question.html

In [10]:
import nest_asyncio
nest_asyncio.apply()

In [11]:
from llama_index import SimpleDirectoryReader, LLMPredictor, ServiceContext, VectorStoreIndex
from llama_index.response.pprint_utils import pprint_response
from langchain import OpenAI

from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

In [12]:
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=-1, streaming=True))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

In [13]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 4.2 MB/s eta 0:00:00


In [50]:
ls gpt_index/docs/examples/data/10q/uber_10q_sept_2022.pdf

gpt_index/docs/examples/data/10q/uber_10q_sept_2022.pdf


In [16]:
march_2022 = SimpleDirectoryReader(input_files=["gpt_index/docs/examples/data/10q/uber_10q_march_2022.pdf"]).load_data()
june_2022 = SimpleDirectoryReader(input_files=["gpt_index/docs/examples/data/10q/uber_10q_june_2022.pdf"]).load_data()
sept_2022 = SimpleDirectoryReader(input_files=["gpt_index/docs/examples/data/10q/uber_10q_sept_2022.pdf"]).load_data()


In [17]:
march_index = VectorStoreIndex.from_documents(march_2022)
june_index = VectorStoreIndex.from_documents(june_2022)
sept_index = VectorStoreIndex.from_documents(sept_2022)

In [18]:
march_engine = march_index.as_query_engine(similarity_top_k=3)
june_engine = june_index.as_query_engine(similarity_top_k=3)
sept_engine = sept_index.as_query_engine(similarity_top_k=3)

In [19]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=sept_engine,
        metadata=ToolMetadata(name='sept_22', description='Provides information about Uber quarterly financials ending September 2022')
    ),
    QueryEngineTool(
        query_engine=june_engine,
        metadata=ToolMetadata(name='june_22', description='Provides information about Uber quarterly financials ending June 2022')
    ),
    QueryEngineTool(
        query_engine=march_engine,
        metadata=ToolMetadata(name='march_22', description='Provides information about Uber quarterly financials ending March 2022')
    ),
]

In [20]:
# Given a query, this query engine `SubQuestionQueryEngine ` will generate a “query plan”
# containing sub-queries against sub-documents before synthesizing the final answer.
s_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

In [21]:
response = s_engine.query('Analyze Uber revenue growth over the latest two quarter filings')


Generated 2 sub questions.
[sept_22] Q: What is the revenue for Uber in the September 2022 quarter?
[june_22] Q: What is the revenue for Uber in the June 2022 quarter?
[june_22] A: The revenue for Uber in the June 2022 quarter is $8,073 million.
[sept_22] A: The revenue for Uber in the September 2022 quarter is $8.3 billion.


In [22]:
print(response)

Uber's revenue has shown growth over the latest two quarter filings. In the September 2022 quarter, the revenue was $8.3 billion, while in the June 2022 quarter, the revenue was $8,073 million. This indicates an increase in revenue between the two quarters.


# Router

define a custom router query engine that can route to either a SQL database or a vector database.

Source: https://gpt-index.readthedocs.io/en/latest/examples/query_engine/SQLRouterQueryEngine.html

In [23]:
import nest_asyncio
nest_asyncio.apply()
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    SQLStructStoreIndex,
    SQLDatabase,
    WikipediaReader
)

In [24]:
# Create Database Schema + Test Data
# Here we introduce a toy scenario where there are 100 tables (too big to fit into the prompt)

from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, select, column
engine = create_engine("sqlite:///:memory:", future=True)
metadata_obj = MetaData()
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)

metadata_obj.create_all(engine)
# print tables
metadata_obj.tables.keys()


dict_keys(['city_stats'])

In [25]:
from sqlalchemy import insert
rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {"city_name": "Berlin", "population": 3645000, "country": "Germany"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.connect() as connection:
        cursor = connection.execute(stmt)
        connection.commit()

In [26]:
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM city_stats")
    print(cursor.fetchall())

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Berlin', 3645000, 'Germany')]


In [27]:
# Load Data
# We first show how to convert a Document into a set of Nodes, and insert into a DocumentStore.
cities = ['Toronto', 'Berlin', 'Tokyo']
wiki_docs = WikipediaReader().load_data(pages=cities)

In [28]:
# Build SQL Index
sql_database = SQLDatabase(engine, include_tables=["city_stats"])
sql_index = SQLStructStoreIndex.from_documents(
    [],
    sql_database=sql_database,
    table_name="city_stats",
)

In [29]:
# Build Vector Index
# build a separate vector index per city
# You could also choose to define a single vector index across all docs, and annotate each chunk by metadata
vector_indices = []
for wiki_doc in wiki_docs:
    vector_index = VectorStoreIndex.from_documents([wiki_doc])
    vector_indices.append(vector_index)

In [30]:
# Define Query Engines, Set as Tools
sql_query_engine = sql_index.as_query_engine()
vector_query_engines = [index.as_query_engine() for index in vector_indices]
from llama_index.tools.query_engine import QueryEngineTool


sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        'Useful for translating a natural language query into a SQL query over a table containing: '
        'city_stats, containing the population/country of each city'
    )
)
vector_tools = []
for city, query_engine in zip(cities, vector_query_engines):
    vector_tool = QueryEngineTool.from_defaults(
        query_engine=query_engine,
        description=f'Useful for answering semantic questions about {city}',
    )
    vector_tools.append(vector_tool)

In [31]:
# Define Router Query Engine
from llama_index.query_engine.router_query_engine import RouterQueryEngine
from llama_index.selectors.llm_selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=([sql_tool] + vector_tools)
)
response = query_engine.query('Which city has the highest population?')

In [32]:
response

Response(response='The city with the highest population is Tokyo.', source_nodes=[], metadata={'result': [('Tokyo',)], 'sql_query': 'SELECT city_name FROM city_stats ORDER BY population DESC LIMIT 1;'})

In [33]:
print(str(response))

The city with the highest population is Tokyo.


In [34]:
response = query_engine.query('Tell me about the historical museums in Berlin')


In [35]:
response

Response(response='Berlin is home to a number of historical museums. One of the most notable is the Deutsches Historisches Museum, which reopened in the Zeughaus and provides an overview of German history spanning over a thousand years. The Jewish Museum has a standing exhibition on two millennia of German-Jewish history. The Allied Museum in Dahlem showcases the history of the Allied forces in Berlin during the Cold War. Additionally, the Stasi Museum, located on the grounds of the former East German Ministry for State Security, offers insights into the history of the Stasi and the division of Berlin. These museums provide visitors with a deeper understanding of the historical events and cultural heritage of Berlin.', source_nodes=[NodeWithScore(node=TextNode(id_='16a6aca1-d2ee-4d73-91e8-d09232ab8a31', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='8ba47f6e-1cb5-468e-b

In [36]:
print(str(response))

Berlin is home to a number of historical museums. One of the most notable is the Deutsches Historisches Museum, which reopened in the Zeughaus and provides an overview of German history spanning over a thousand years. The Jewish Museum has a standing exhibition on two millennia of German-Jewish history. The Allied Museum in Dahlem showcases the history of the Allied forces in Berlin during the Cold War. Additionally, the Stasi Museum, located on the grounds of the former East German Ministry for State Security, offers insights into the history of the Stasi and the division of Berlin. These museums provide visitors with a deeper understanding of the historical events and cultural heritage of Berlin.
